# Crunchers

In this notebook we demonstrate crunchers, silicone's most fundamental infilling methods. You will need to install silicone from https://github.com/znicholls/silicone/ in order to run this code. 

## Imports

In [1]:
# NBVAL_IGNORE_OUTPUT
import os.path
import traceback

import pandas as pd
import pyam
import matplotlib.pyplot as plt
import numpy as np

import silicone.database_crunchers
from silicone.utils import (
    _get_unit_of_variable,
    find_matching_scenarios,
    _make_interpolator,
    _make_wide_db
)

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


## Constants

In [2]:
SR15_SCENARIOS = "./sr15_scenarios.csv"

## Example data

Here we pull some example data by downloading a selection of the SR1.5 scenarios.

In [3]:
def get_sr15_scenarios(output_file):
    conn = pyam.iiasa.Connection("iamc15")
    valid_model_ids = [
        "MESSAGE*",
        "AIM*",
        "C-ROADS*",
        "GCAM*",
        #"IEA*",
        #"IMAGE*",
        #"MERGE*",
        #"POLES*",
        #"REMIND*",
        "WITCH*"
    ]
    variables_to_fetch = [
        "Emissions*",
    ]
    for model in valid_model_ids:
        print("Fetching data for {}".format(model))
        for variable in variables_to_fetch:
            print("Fetching {}".format(variable))
            var_df = conn.query(model=model, variable=variable, region='World')
            try:
                df.append(var_df, inplace=True)
            except NameError:
                df = pyam.IamDataFrame(var_df)

    print("Writing to {}".format(output_file))
    df.to_csv(output_file)

In [4]:
# NBVAL_IGNORE_OUTPUT
if not os.path.isfile(SR15_SCENARIOS):
    get_sr15_scenarios(SR15_SCENARIOS)

### Starting point

Our starting point is the test data, loaded a `pyam.IamDataFrame`.

In [ ]:
# NBVAL_IGNORE_OUTPUT
sr15_data = pyam.IamDataFrame(SR15_SCENARIOS)
sr15_data.timeseries().head()

pyam.utils - INFO: Reading `./sr15_scenarios.csv`


## Crunchers

Silicone's 'crunchers' are used to determine the relationship between a 'follower variable' and 'lead variable(s)' from a given database. The 'follower variable' is the variable for which we want to generate data e.g. `Emissions|C3F8` while the 'lead variable(s)' is the variable we want to use in order to infer a timeseries of the 'follower variable'. The lead variable is typically CO$_2$ but might be a more complex combination of gases e.g. CO$_2$, CH$_4$ and N$_2$O. 

Each cruncher has a `derive_relationship` method. Its docstring describes what it does.

In [ ]:
print(silicone.database_crunchers.base._DatabaseCruncher.derive_relationship.__doc__)
# # an alternative interface is shown if you use the line below instead
# silicone.database_crunchers.base._DatabaseCruncher.derive_relationship?

These crunchers are best explored by looking at the examples below.

### Closest RMS cruncher

This cruncher uses the 'closest RMS' methodology. It is described by its docstring.

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherRMSClosest.__doc__)
# silicone.database_crunchers.DatabaseCruncherRMSClosest?

#### Infilling

Firstly, let's cut the database down to a size that is comprehensible.

In [ ]:
# NBVAL_IGNORE_OUTPUT
sr15_data_closest_rms = sr15_data.filter(model=["WITCH-GLOBIOM 4.2"])

Now we can crunch the relationship between e.g. `Emissions|CO2` and `Emissions|VOC`.

In [ ]:
# NBVAL_IGNORE_OUTPUT
cruncher = silicone.database_crunchers.DatabaseCruncherRMSClosest(sr15_data_closest_rms)

In [ ]:
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"])
print(filler.__doc__)
# filler info:

Now we can do some infilling. As a sanity check, we firstly make sure that if we pass in a CO$_2$ timeseries which is already in the database, we get back its VOC emissions timeseries. 

In [ ]:
example_model_scen = {
    "model": "WITCH-GLOBIOM 4.2",
    "scenario": "ADVANCE_INDC",
}
example_input = sr15_data_closest_rms.filter(**example_model_scen).data
example_input["model"] = "example"
example_input["scenario"] = "example"
example_input = pyam.IamDataFrame(example_input)

In [ ]:
# NBVAL_IGNORE_OUTPUT
example_input_filled = filler(example_input)

In [ ]:
#NBVAL_IGNORE_OUTPUT
example_input_filled.timeseries()

In [ ]:
#NBVAL_IGNORE_OUTPUT
sr15_data_closest_rms.filter(variable="Emissions|VOC", **example_model_scen).timeseries()

As expected, the cruncher has picked out the scenario which matches and returned its VOC timeseries. This can also be seen in a plot.

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_closest_rms.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
# sr15_data_closest_rms.filter(variable="*CO2", **example_model_scen).line_plot(ax=ax, color="black")
example_input.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_closest_rms.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
example_input_filled.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

Now we can use our filler to infill other timeseries.

In [ ]:
filler_input = sr15_data.filter(model="MESSAGEix-GLOBIOM 1.0", scenario="CD-LINKS_NPi2020_1600")

This scenario is missing `Emissions|VOC`.

In [ ]:
#NBVAL_IGNORE_OUTPUT
filler_input.filter(variable="Emissions|VOC").data

In [ ]:
filler_input_filled = filler(filler_input)

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_closest_rms.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
# sr15_data_closest_rms.filter(variable="*CO2", **example_model_scen).line_plot(ax=ax, color="black")
filler_input.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_closest_rms.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
filler_input_filled.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

As we can see, on an RMS basis our input timeseries is closest to the 'ADVANCE_2030_Med2C' scenario and hence its `Emissions|VOC` pathway is returned.

In [ ]:
#NBVAL_IGNORE_OUTPUT
filler_input_filled.timeseries()

In [ ]:
#NBVAL_IGNORE_OUTPUT
sr15_data_closest_rms.filter(scenario="ADVANCE_2030_Med2C", variable="Emissions|VOC").timeseries()

### Rolling Window Quantile cruncher

This cruncher uses the 'rolling window quantile' methodology. It is described by its docstring.

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherQuantileRollingWindows.__doc__)
# silicone.database_crunchers.DatabaseCruncherQuantileRollingWindows?

#### Infilling

Because it relies on quantile analysis, this database does best with large amounts of data. Hence we keep quite a lot of emissions timeseries before beginning.

In [ ]:
sr15_data_quantile = sr15_data.copy().filter(
    model=["WITCH-GLOBIOM 4.2"], variable=["Emissions|CO2", "Emissions|VOC"]
)

In [ ]:
cruncher = silicone.database_crunchers.DatabaseCruncherQuantileRollingWindows(sr15_data_quantile)

In [ ]:
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"])
print(filler.__doc__)
# filler?

Now we can infill.

In [ ]:
filler_data = sr15_data.filter(
    model="MESSAGEix-GLOBIOM 1.0", 
    scenario="CD-LINKS_NPi2020_1600", 
    variable=["Emissions|CO2"]
)

The `filler_data` is missing `Emissions|VOC`.

In [ ]:
#NBVAL_IGNORE_OUTPUT
filler_data.filter(variable="Emissions|VOC").data

The rolling windows quantile can only downscale on the timesteps in the database. If this isn't the case, we will receive a `ValueError`.

In [ ]:
try:
    filler(filler_data)
except ValueError:
    traceback.print_exc(limit=0, chain=False)

We can fix this by filtering first.

In [ ]:
# NBVAL_IGNORE_OUTPUT
filler_data = filler_data.filter(year=sr15_data_quantile["year"].unique())

In [ ]:
#NBVAL_IGNORE_OUTPUT
filler_data.timeseries()

In [ ]:
#NBVAL_IGNORE_OUTPUT
filler_data_filled = filler(filler_data)
filler_data_filled.head()

In [ ]:
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_quantile.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
# sr15_data_closest_rms.filter(variable="*CO2", **example_model_scen).line_plot(ax=ax, color="black")
filler_data.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_quantile.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
filler_data_filled.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

#### Recovering the data you put in

Because of the averaging mechanism, the cruncher will only precisely reproduce the input data when only one timeseries is put into it. With multiple data series, this is not guaranteed, although will happen in the limit case of many windows. 

This section does not illustrate new functionality, merely investigates the mathematical effects of changing the number of windows.

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Multiple data are input - the 50th percentile and nwindows=10 is the default
filler_data = sr15_data_quantile.filter(scenario='ADVANCE_2030_Med2C')
self_cruncher = silicone.database_crunchers.DatabaseCruncherQuantileRollingWindows(sr15_data)
self_filler_default = self_cruncher.derive_relationship("Emissions|VOC", ["Emissions|VOC"], nwindows=10)
infilled_quantile = self_filler_default(filler_data)
infilled_quantile.head()

The default behaviour shows the correct trend, but the windows-based interpolation introduces some noise.

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot()
filler_data.filter(variable='Emissions|VOC', ).line_plot(ax=ax, **pkwargs)
infilled_quantile.filter(variable='Emissions|VOC').line_plot(ax=ax, color="black", linestyle="--", dashes=(8, 10))

By increasing the windows number, we can get a higher fidelity - at the cost of more computational power and less averaging over different datapoints. 

In [ ]:
# NBVAL_IGNORE_OUTPUT
# With a lower window number, the deviations become clearer
self_filler_low_w = self_cruncher.derive_relationship("Emissions|VOC", ["Emissions|VOC"], nwindows=2)
infilled_low_w = self_filler_low_w(filler_data)
infilled_low_w.head()

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Higher window numbers mean that we weight data with the exact same x-value more highly, recovering the original relation
self_filler_high_w = self_cruncher.derive_relationship("Emissions|VOC", ["Emissions|VOC"], nwindows=30)
infilled_high_w = self_filler_high_w(filler_data)
infilled_high_w.head()

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Here the lower window count (orange) results in large deviations whereas the high window case (red) provides a close match. 
pkwargs = {"color": "scenario", "linewidth": 2}
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
# This line illustrates the background data informing the deviations (but confuses the plots). 
# Re-add it in place of the legend line if desired
#sr15_data.filter(variable='Emissions|VOC',).line_plot(ax=ax, linewidth=0.1)

filler_data.filter(variable='Emissions|VOC', ).line_plot(ax=ax, **pkwargs)
infilled_quantile.filter(variable='Emissions|VOC').line_plot(ax=ax, color="black", linestyle="--", dashes=(8, 10))
infilled_low_w.line_plot(ax=ax, color="blue", linestyle="--", dashes=(7, 8))
infilled_high_w.line_plot(ax=ax, color="red", linestyle="--", dashes=(7, 8))
plt.legend(['Original', 'n=10', 'n=2', 'n=30'])

### Time-dependent ratio cruncher
This cruncher uses the 'time-dependent ratio' methodology. It is described by its docstring.

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherTimeDepRatio.__doc__)

#### Infilling
This technique is extracts only a single constant multiplier from all data fed in to the cruncher, so additional data does not radically change how it behaves. We will use the same amount of data as in the previous case for comparison. This cruncher works well for cases like splitting groups of gases into constituent parts, but poorly for gases which may have negative emissions. 

In [ ]:
sr15_data_tdr = sr15_data.copy().filter(
    model=["WITCH-GLOBIOM 4.2"], 
    variable=["Emissions|CO2", "Emissions|VOC", "Emissions|CH4" ,"Emissions|HCF*"]
)

In [ ]:
cruncher = silicone.database_crunchers.DatabaseCruncherTimeDepRatio(sr15_data_tdr)
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"])
print(filler.__doc__)

In [ ]:
# NBVAL_IGNORE_OUTPUT
filler_data = sr15_data.filter(
    model="MESSAGEix-GLOBIOM 1.0", 
    scenario="CD-LINKS_NPi2020_1600", 
    variable=["Emissions|CO2"]
)
filler_data = filler_data.filter(year=sr15_data_quantile["year"].unique())
filler_data.data

In [ ]:
# NBVAL_IGNORE_OUTPUT
results = filler(filler_data)
results.head()

We observe that the results become somewhat odd when the lead emissions may be negative. By default, we only use data where the lead has the same sign as the follow, so some of the pathways are discounted when their lead goes negative but the infillee data hasn't. This can be switched off using the `same_sign = False` option. 

In [ ]:
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
sr15_data_tdr.filter(variable="*CO2").line_plot(ax=ax, **pkwargs)
filler_data.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database CO$_2$ and lead emissions timeseries")

ax = fig.add_subplot(122)
sr15_data_tdr.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
results.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

By comparison, with `same_sign = False`, we obtain unphysically negative results for VOC:

In [ ]:
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"], same_sign = False)
results = filler(filler_data)
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(111)
sr15_data_tdr.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
results.filter(variable="*VOC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database VOC and filled follower emissions timeseries")

plt.tight_layout()

#### HFC emission splitting
One of the features of the time-dependent ratio is that if the constituent elements consistently add up to a particular value, the infilled results will too, i.e. if for gases A, B, C we have A = B + C in all source material, then this will hold for the infilled values too. This makes it particularly appropriate for splitting gases into their constituents. 

In [ ]:
sr15_data_hfc = sr15_data.filter(variable="Emissions|HFC*")
sr15_data_hfc.variables()

In [ ]:
# We don't have all variables for all scenarios, but by finding a scenario where we do we can make progress
common_models = set(sr15_data_hfc.filter(variable="Emissions|HFC|HFC245fa").models()).intersection(
    set(sr15_data_hfc.filter(variable="Emissions|HFC").models())
)
common_scenarios = set(sr15_data_hfc.filter(variable="Emissions|HFC|HFC245fa", model=common_models).scenarios()).intersection(
    set(sr15_data_hfc.filter(variable="Emissions|HFC", model=common_models).scenarios())
)
infill_scenarios = "GEA_Eff_1p5C"
#set(sr15_data_hfc.filter(variable="Emissions|HFC").scenarios()) - set(sr15_data_hfc.filter(variable="Emissions|HFC|HFC245fa").scenarios())

infill_models ='MESSAGE V.3' 
# set(sr15_data_hfc.filter(variable="Emissions|HFC").models()) - set(sr15_data_hfc.filter(variable="Emissions|HFC|HFC245fa").models())

infill_scenarios

In [ ]:
common_years = np.arange(2010, 2105, 10)
hfc_training_data = sr15_data_hfc.filter(model=common_models, scenario=common_scenarios, year=common_years)
cruncher = silicone.database_crunchers.DatabaseCruncherTimeDepRatio(hfc_training_data)
filler = cruncher.derive_relationship("Emissions|HFC|HFC245fa", ["Emissions|HFC"])

In [ ]:
#NBVAL_IGNORE_OUTPUT
to_infill = sr15_data_hfc.filter(model=infill_models, scenario=infill_scenarios, year=common_years)
infilled = filler(to_infill)
infilled.head()

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
hfc_training_data.filter(variable="*HFC").line_plot(ax=ax, **pkwargs)
to_infill.filter(variable="*HFC").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database HFC and lead emissions timeseries")

ax = fig.add_subplot(122)
hfc_training_data.filter(variable="*HFC245fa").line_plot(ax=ax, **pkwargs)
infilled.filter(variable="*HFC245fa").line_plot(ax=ax, color="black", linestyle="--", dashes=(10, 15))
ax.set_title("Database HFC245fa and filled follower emissions timeseries")

plt.tight_layout()

### Constant ratio cruncher
This infiller simply multiples a known value by a constant, which it must be told. It must also be told the units of the value it returns. It does not require an input dataframe. 

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherTimeDepRatio.__doc__)

In [ ]:
cruncher = silicone.database_crunchers.DatabaseCruncherConstantRatio()
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"], 0.5, "kt VOC/yr")
print(filler.__doc__)

In [ ]:
#NBVAL_IGNORE_OUTPUT
to_infill = sr15_data.filter(model=infill_models)
infilled = filler(to_infill)
infilled.head()

The values returned are simply the input, scaled by a factor. 

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)
infilled.line_plot(ax=ax, **pkwargs)
to_infill.filter(variable="*CO2").line_plot(ax=ax, color="black", linestyle="--", dashes=(1, 1))
ax.set_title("Database HFC and lead emissions timeseries")


### Interpolate specified scenarios and models

This cruncher pre-filters the data to such that it only uses data from scenarios or models containing a specified string (typically scenarios which start with "SSP"). 

In [ ]:
print(silicone.database_crunchers.DatabaseCruncherScenarioAndModelSpecificInterpolate.__doc__)

#### Using only one model

We will show infilling using a database comprised of data only from variants of the MESSAGE model. 

In [ ]:
sr15_data_message = sr15_data.filter(
    model=["MESSAGE*"], 
    variable=["Emissions|CO2", "Emissions|VOC", "Emissions|CH4" ,"Emissions|HCF*"]
)
sr15_data_message.models()

As we can see, variants of the MESSAGE model have submitted data for a large number of scenarios.

In [ ]:
sr15_data_message.scenarios()

For our example, we will infill data using data taken only from variants of the AIM model and we are only going to infill data for variants of SSP2.

In [ ]:
sr15_data_aim_ssp2 = sr15_data.filter(
    model=["AIM*"], 
    scenario="SSP2*",
    variable=["Emissions|CO2", "Emissions|VOC", "Emissions|CH4" ,"Emissions|HCF*"], 
    year=sr15_data_message.data["year"].unique()
)
sr15_data_aim_ssp2.models()

In [ ]:
sr15_data_aim_ssp2.scenarios()

Although the lineshapes for CO$_2$ (left-hand column below) look similar in the AIM and MESSAGE models, the lineshapes for VOC (right-hand column below) are very different. 

In [ ]:
# NBVAL_IGNORE_OUTPUT
pdf = pyam.concat([sr15_data_message, sr15_data_aim_ssp2])

fig = plt.figure(figsize=(16, 9))

ax = fig.add_subplot(121)
pdf.filter(scenario="SSP2*", variable="*CO2").line_plot(ax=ax, color="scenario", linestyle="model")

ax = fig.add_subplot(122)
pdf.filter(scenario="SSP2*", variable="*VOC").line_plot(ax=ax, color="scenario", linestyle="model")

plt.tight_layout()

Variants of the MESSAGE model have submitted data for many scenarios, but this cruncher allows us to specify that only data from scenarios which start with SSP2 should be used. 

In [ ]:
cruncher = silicone.database_crunchers.DatabaseCruncherScenarioAndModelSpecificInterpolate(sr15_data_message)
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"], required_scenario="SSP2*")
infilled = filler(sr15_data_aim_ssp2)

We can now compare the original data, the database which was used to derive the infilling relationship and the infilled data. We see that the infilled results result in timeseries which much more strongly resemble the timeseries in the database (i.e. the similarity in CO$_2$ emissions is now also reflected in VOC emissions). We also see that no emissions values more extreme than the range of values contained in the input database are returned by this cruncher.

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}
fig = plt.figure(figsize=(16, 9))

ax = fig.add_subplot(131)
sr15_data_aim_ssp2.filter(variable="*VOC").line_plot(ax=ax, **pkwargs)
ax.set_title("Original (AIM) timeseries")

ax = fig.add_subplot(132)
infilled.line_plot(ax=ax, **pkwargs)
ax.set_title("Infilled timeseries\n(infilling AIM results based on the relationship\nbetween CO2 and VOC in MESSAGE)")

ax = fig.add_subplot(133)
sr15_data_message.filter(scenario="SSP2*", variable="*VOC").line_plot(ax=ax, color="scenario")
ax.set_title("Database (MESSAGE) timeseries")

plt.tight_layout()

#### Multiple models

However, if we include data from many different models in our database, the inter-model differences often exceed the differences within a single model's data, making for rather chaotic interpolation. This is shown below.

In [ ]:
# NBVAL_IGNORE_OUTPUT
scenario_string = "SSP2*"
too_much_data = (
    sr15_data.filter(
        variable=["Emissions|CO2", "Emissions|VOC", "Emissions|CH4" ,"Emissions|HCF*"]
    ).filter(
        model=["AIM*"], 
        keep=False
    )
)

cruncher = silicone.database_crunchers.DatabaseCruncherScenarioAndModelSpecificInterpolate(too_much_data)
filler = cruncher.derive_relationship("Emissions|VOC", ["Emissions|CO2"], required_scenario=scenario_string)

infilled = filler(sr15_data_aim_ssp2)

wide_db = _make_wide_db(too_much_data.filter(scenario=scenario_string))
interpolator = _make_interpolator("Emissions|VOC", ["Emissions|CO2"], wide_db, "year")

There is no simple function relating the CO$_2$ and VOC trends in this case.

In [ ]:
# NBVAL_IGNORE_OUTPUT
fig = plt.figure(figsize=(16, 9))

ax = fig.add_subplot(121)
too_much_data.filter(
    scenario=scenario_string, variable="*VOC"
).line_plot(
    ax=ax, color="scenario", linestyle="model"
)
ax.set_title("VOC trends in the cruncher's database")

ax = fig.add_subplot(122)
plt.scatter(
    wide_db["Emissions|CO2"].loc[wide_db.index.get_level_values('year')==2060], 
    wide_db["Emissions|VOC"].loc[wide_db.index.get_level_values('year')==2060],
)
plt.xlabel("Mt CO2/yr")
plt.ylabel("Mt VOC/yr")
ax.set_title("VOC vs CO2 in 2060")

As a result, we have uneven and non-monotonic trends in our inferred relationship between the lead and follower gases (here CO$_2$ and VOC respectively).

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(121)

xs = np.arange(-60000, 150000)
plt.plot(xs, interpolator[2060](xs))
plt.title("VOC interpolator in 2060")
plt.xlabel("Mt CO2/yr")
plt.ylabel("Mt VOC/yr")

In [ ]:
# NBVAL_IGNORE_OUTPUT
pkwargs = {"color": "scenario", "linewidth": 2}
fig = plt.figure(figsize=(16, 9))

ax = fig.add_subplot(121)
too_much_data.filter(scenario="SSP2*", variable="*VOC").line_plot(ax=ax, color="scenario")
ax.set_title("VOC trends in database")

ax = fig.add_subplot(122)
infilled.line_plot(ax=ax, **pkwargs)
ax.set_title("Infilled VOC trends")

This package also has a function for detecting which of a group of scenarios creates an interpolation that best matches a dataset. Using this tool, we see that the CO$_2$-CH$_4$ relations in some SSP2 scenarios in the AIM data are more similar to SSP3/SSP1 data from MESSAGE models. 

In [ ]:
data_to_classify_19 = sr15_data_aim_ssp2.filter(scenario="SSP2-19")
possible_ssps = ["SSP1*", "SSP2*", "SSP3*"]
find_matching_scenarios(sr15_data_message, data_to_classify_19, "Emissions|CH4", ["Emissions|CO2"], possible_ssps)

In [ ]:
find_matching_scenarios(
    sr15_data_message,
    sr15_data_aim_ssp2.filter(scenario="SSP2-Baseline"), 
    "Emissions|CH4", 
    ["Emissions|CO2"], 
    possible_ssps
)

In [ ]:
find_matching_scenarios(
    sr15_data_message,
    sr15_data_aim_ssp2.filter(scenario="SSP2-60"), 
    "Emissions|CH4", 
    ["Emissions|CO2"], 
    possible_ssps
)

In [ ]:
find_matching_scenarios(
    sr15_data_message,
    sr15_data_aim_ssp2.filter(scenario="SSP2-34"), 
    "Emissions|CH4", 
    ["Emissions|CO2"],
    possible_ssps
)

In [ ]:
sr15_data_message.models()

It's also possible to break down results by models and scenarios at the same time, and to get the results back in numerical form (they are reported in increasing order of distance, so the top result is the closest). In the event that a model/scenario combination doesn't have any data, a warning is displayed and the 'distance' is reported as infinity. 

In [ ]:
# NBVAL_IGNORE_OUTPUT
possible_models = sr15_data_message.models()
find_matching_scenarios(
    sr15_data_message,
    sr15_data_aim_ssp2.filter(scenario="SSP2-Baseline"), 
    "Emissions|CH4", 
    ["Emissions|CO2"], 
    possible_ssps, 
    possible_models, 
    return_all_info=True
)

Here we see that the preference for SSP3 is fairly substantial in this case, and that data is only found for these scenarios in the MESSAGE-GLOBIOM model. 

In some cases, we may wish to ignore the initial differences and only look for the closest trendlines, i.e. match the differentials. This is equivalent to setting all initial values to the same number, then performing the above analysis. This can be done by setting the "use_change_not_abs" value to True:

In [ ]:
# NBVAL_IGNORE_OUTPUT
possible_models = sr15_data_message.models()
find_matching_scenarios(
    sr15_data_message,
    sr15_data_aim_ssp2.filter(scenario="SSP2-Baseline"), 
    "Emissions|CH4", 
    ["Emissions|CO2"], 
    possible_ssps, 
    possible_models, 
    return_all_info=True, use_change_not_abs=True
)